In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


In [49]:


data = pd.read_csv('data/CondMat.csv')
print(data.shape)
data.head()



(182684, 15)


,node_1,node_2,neighbours_1,neighbours_2,density_ego_with_node_1,density_ego_without_node_1,density_ego_with_node_2,density_ego_without_node_2,common_neigbhours,total_neigbhours,jaccard_coefficient,prefferential_attachment,friends_measure,shortest_path,link_exists
0,81626.0,11894.0,36.0,2.0,0.117117,0.066667,1.000000,1.000000,1.0,37.0,0.027027,72.0,42.0,1.0,1.0
1,81626.0,16246.0,36.0,42.0,0.117117,0.066667,0.171650,0.131243,5.0,73.0,0.068493,1512.0,95.0,1.0,1.0
2,81626.0,16322.0,36.0,4.0,0.117117,0.066667,0.800000,0.666667,1.0,39.0,0.025641,144.0,44.0,1.0,1.0
3,81626.0,18914.0,36.0,6.0,0.117117,0.066667,0.523810,0.333333,2.0,40.0,0.050000,216.0,51.0,1.0,1.0
4,81626.0,19729.0,36.0,35.0,0.117117,0.066667,0.157143,0.107563,7.0,64.0,0.109375,1260.0,125.0,1.0,1.0


In [50]:
data = data.drop(['node_1', 'node_2'], axis = 'columns')
label = data['link_exists']
train_data = data.drop(['link_exists'], axis = 'columns')
print(train_data.shape)
print(label.shape)

(182684, 12)
(182684,)


In [29]:
x_train, x_test, y_train, y_test = train_test_split(train_data, label, test_size=0.2, random_state=1)

In [51]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

model = GaussianNB()
# model = DecisionTreeClassifier(criterion='entropy', random_state=1, max_leaf_nodes=5)
# model = AdaBoostClassifier(algorithm='SAMME')
# model = BaggingClassifier()


model.fit(x_train, y_train)
acc_score_train = accuracy_score(y_train, model.predict(x_train))
acc_score_test = accuracy_score(y_test, model.predict(x_test))

print(f'Accuracy during training: {acc_score_train}') 
print(f'Accuracy during testing: {acc_score_test}')

Accuracy during training: 0.966719419102588
Accuracy during testing: 0.9676098287416233


In [52]:
import torch
from torch import nn

print(torch.cuda.is_available())

print(torch.cuda.device_count())


True
1


In [53]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [32]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(12, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [33]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=12, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [54]:
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
dataset = TensorDataset(pd.DataFrame.to_numpy(x_train, dtype=np.float64), pd.DataFrame.to_numpy(y_train, dtype=np.float64))
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

AttributeError: 'SingleBlockManager' object has no attribute 'as_array'

In [42]:
model = NeuralNetwork()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [47]:
# Training loop
for epoch in range(4):
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{4}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    # Example test data
    test_X = torch.tensor(x_test.values, dtype=torch.float32)
    test_y = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

    test_outputs = model(test_X)
    predicted = (test_outputs > 0.5).float()
    accuracy = (predicted == test_y).sum() / test_y.size(0)

    print(f'Accuracy: {accuracy:.4f}')

Epoch [1/4], Loss: 0.0008
Epoch [2/4], Loss: 0.0009
Epoch [3/4], Loss: 0.0020
Epoch [4/4], Loss: 0.0007
Accuracy: 0.9985
